<center> <h1> Automatic Differentiation for Solid Mechanics in Julia</h1> </center>
<br>
<center> <h2> Andrea Vigliotti</h2> </center>

<center> <h2> Juliacon$^{2022}$, July 27$^{\textrm{th}}$ - 29$^{\textrm{th}}$, 2022 <h2> </center>

<table><tr></tr>
<tr>
<td> <img src="./figures/cira_logo.jpg" alt="Drawing" style="height: 100px;"/> </td>
    <td>  </td>
<td> <img src="./figures/logo_juliacon2022.jpg" alt="Drawing" style="height: 100px;"/> </td>
</tr></table>

<h1> Automatic Differentiation for Solid Mechanics in Julia </h1>

<br>
<br>
<ul>
<li><h2> Why automatic differentiation for solid mechanics ?</h2></li>
<br>    
<li><h2> Why Julia ?</h2></li>
<br>    
<li><h2> AD4SM.jl and some examples </h2></li>
</ul> 

<h1> Automatic Differentiation for Solid Mechanics in Julia </h1>

<br>
<br>
<ul>
<li><h2> Why automatic differentiation for solid mechanics ?</h2></li>
<br>    
<li><h2> What type of AD ?</h2></li>
<br>    
<li><h2> Why Julia ?</h2></li>
<br>    
<li><h2> AD4SM.jl and some examples </h2></li>
</ul> 

<h3><center> The <it>traditional</it> implementation of the Finite Element Method </center> </h3>
Cauchy Equilibrium
\begin{align*}
\sigma_{ij,j} + f_i &=0 \qquad \text{in} \quad V \\
\sigma_{ij} n_j - g_i &=0 \qquad \text{on}\quad S
\end{align*}

<h3><center> The <it>traditional</it> implementation of the Finite Element Method </center> </h3>

the Virtual Work Principle is
\begin{equation*}\label{VirtualWork}
\int_{V_0} \left(P_{ij}\,\frac{\partial F_{ij}}{\partial u_k}-f_{0_k}\right)\, \delta u_k\, \mathrm{d} V_0 -\int_{S_0} g_{0_k}\, \delta u_k\, \mathrm{d}S_0=0 \qquad \begin{aligned}
&\forall\, \delta u_k\\[5pt]
&\ k = 1,2,3
\end{aligned} 
\end{equation*}

the residual force vector is
\begin{equation*}
\mathbf{r}=\sum_{k=1}^{N_{BE}} \sum_{l=1}^{N_{BW}^k} w_l^k\, \left[P_{ij}\frac{\partial 
    F_{ij}}{\partial \mathbf{u}} - \mathbf{f}_0 \right]_{r^k_l} -
\sum_{k=1}^{N_{SE}} \sum_{l=1}^{N_{SW}^k} v_l^k\, \left[\mathbf{g}_0\right]_{r^k_l}=\mathbf{0}
\end{equation*}

the tangent stiffness matrix is
\begin{equation*}
\begin{split}
\frac{\partial \mathbf{r}}{\partial \mathbf{u}}= \sum_{k=1}^{N_{BE}} \sum_{l=1}^{N_{BW}^k} w_l^k\, 
\left[\frac{\partial P_{ij}}{\partial F_{hk}}\frac{\partial F_{hk}}{\partial \mathbf{u}}\frac{\partial 
    F_{ij}}{\partial \mathbf{u}}  - 
\frac{\partial \mathbf{f}_0}{\partial \mathbf{u}}\right]_{r^k_l}  - \sum_{k=1}^{N_{SE}} 
\sum_{l=1}^{N_{SW}^k} v_l^k\, \left[\frac{\partial \mathbf{g}_0}{\partial \mathbf{u}}\right]_{r^k_l}
\end{split}
\end{equation*}

<h3><center> Free energy minimization principle </center></h3>
    
\begin{equation*}
    \delta U = 0 \qquad \text{with} \qquad U = \int_{V_0} \left(\phi-b_0\right) \,\mathrm{d}V_0 - \int_{S_0} t_0 \,\mathrm{d}S_0\, 
\end{equation*}

we can use FE discretization for evaluating $U$
\begin{equation*}
    U(\mathbf{u}) = \sum_{k=1}^{N_{BE}} \sum_{l=1}^{N_{BW}^k} w_l^k \left[\phi + b_{0}\right]_{r^k_l} + 
    \sum_{k=1}^{N_{SE}} \sum_{l=1}^{N_{SW}^k} v_l^k\, \left[t_0\right]_{r^k_l}
\end{equation*}

the residual force vector is 
\begin{equation*}
\mathbf{r} = \frac{\partial\, U}{\partial \mathbf{u}} =0 
\end{equation*}

the tangent stiffness matrix is
\begin{equation*}
	\frac{\partial \mathbf{r}}{\partial \mathbf{u}} = \frac{\partial^2\, U}{\partial \mathbf{u} \partial \mathbf{u}}
\end{equation*}

In the presence of complex boundary conditions and constraints we use AD with Lagrange Multipliers

\begin{equation*}
L\left(\mathbf{u}, \mathbf{\lambda}\right) = U\left(\mathbf{u}\right) - \mathbf{\lambda}\cdot 
\mathbf{g}\left(\mathbf{u}\right)
\end{equation*}

equilibrium is

\begin{equation*}
\nabla L = \mathbf{0}
\end{equation*}
with:

\begin{align*}
\nabla L &= \left[\begin{matrix}
\dfrac{\partial U}{\partial \mathbf{u}} - \mathbf{\lambda} \cdot \dfrac{\partial \mathbf{g}}{\partial \mathbf{u}}\\[5pt]
-\mathbf{g}
\end{matrix}\right]   %\\[5pt]
\qquad \text{and} \qquad
\nabla^2 L &= \left[\begin{matrix}
\dfrac{\partial^2 U}{\partial \mathbf{u}\partial \mathbf{u}} - \mathbf{\lambda} \cdot \dfrac{\partial^2 \mathbf{g}}{\partial \mathbf{u}\partial \mathbf{u}} & \hspace{20pt}-\dfrac{\partial \mathbf{g}}{\partial \mathbf{u}}^T \\[5pt]
-\dfrac{\partial \mathbf{g}}{\partial \mathbf{u}} & \hspace{20pt}\mathbf{0}
\end{matrix}\right]
\end{align*}

<h2> The rod element </h2>

<h4> the free energy is </h4>
<center>
\begin{equation*}
\phi^\text{rod} = \frac{1}{2}\,E_s\,A\,l_0  \left( \frac{l}{l_0}-1 \right)^2 \qquad \text{with} \qquad \begin{cases} l_0 &= \|\mathbf{r}_2-\mathbf{r}_1\| \\[10pt] l &= \|\mathbf{r}_2-\mathbf{r}_1 + \mathbf{u}_2-\mathbf{u}_1\| \end{cases}
\end{equation*}
</center>    


<h4> the residual force vector and the tangent stiffness matrix are </h4>

<br>
\begin{align*}
\mathbf{r} = \frac{\partial}{\partial \mathbf{u}}\phi^\text{rod} &= E_s A \left(\frac{l}{l_0}-1\right)\frac{\partial l}{\partial \mathbf{u}} \\[5pt]
\mathbf{K}_t = \frac{\partial^2}{\partial \mathbf{u}\partial \mathbf{u}}\phi^\text{rod} &= E_s A \left[\frac{1}{l_0} \frac{\partial l}{\partial \mathbf{u}} \frac{\partial l}{\partial \mathbf{u}} + \left(\frac{l}{l_0}-1\right)\frac{\partial^2 l}{\partial \mathbf{u}\partial \mathbf{u}}\right]
\end{align*}
<br>
where
<br>
$$
\frac{\partial l}{\partial \mathbf{u}} = \frac{1}{l}\left[\begin{matrix}-\Delta r_1\\-\Delta r_2\\-\Delta r_3\\\Delta r_1\\\Delta r_2\\\Delta r_3\end{matrix}\right] 
\qquad \qquad
\frac{\partial^2 l}{\partial \mathbf{u}\partial \mathbf{u}} = \left[\begin{matrix}1&0&0&-1&0&0\\0&1&0&0&-1&0\\0&0&1&0&0&-1\\-1&0&0&1&0&0\\0&-1&0&0&1&0\\0&0&-1&0&0&1\end{matrix}\right]
$$

In [ ]:
function calcFiKt(nodes, u)
  
  l0       = norm(nodes[:,2]-nodes[:,1])
  l        = norm(nodes[:,2]+u[:,2] - (nodes[:,1]+u[:,1]))  
  ϕ        = l0/2 *(l/l0-1)^2

  Δx,Δy,Δz = u[:]
  dldu     = [-Δx, -Δy, -Δz, Δx, Δy, Δz]/l;
  d2ldu2   = [ 1.0 0.0 0.0 -1.0 0.0 0.0;
               0.0 1.0 0.0 0.0 -1.0 0.0; 
               0.0 0.0 1.0 0.0 0.0 -1.0;
               -1.0 0.0 0.0 1.0 0.0 0.0; 
               0.0 -1.0 0.0 0.0 1.0 0.0; 
               0.0 0.0 -1.0 0.0 0.0 1.0]
  dphidl   = (l/l0-1)/l0
  dphi2dl2 = 1/l0^2  
  fact1    = -dphidl/l^2 + dphi2dl2/l; 
  
  r        = dphidl * dldu;
  Kt       = fact1*l*(dldu*transpose(dldu)) + (dphidl/l)*d2ldu2

  (ϕ, r, Kt)
end

In [ ]:
function getϕ(nodes, u)
    
  l0       = norm(nodes[:,2]-nodes[:,1])
  l        = norm(nodes[:,2]+u[:,2] - (nodes[:,1]+u[:,1]))  
  ϕ        = l0/2 *(l/l0-1)^2
end

<center> <h1> What type of automatic differentiation? </h1> </center>



Forward mode AD works well for solid mechanics
<br>

<ul>
<li> Even if a FE model can have millions of degrees of freedom, at each integration point, only the local element DoFs are involved, and in the most common cases
    <ul>
        <li> 2D tria have 6 DoFs </li>
        <li> 2D quads have 6 DoFs </li>
        <li> 3D hexa have 24 DoFs </li>
    </ul>    
    </li>
<li> At integration points the free energy is usually a function of the deformation gradient </li>
<li> The calculation of each element contribution is independent from the others</li>    
<li> First order forward AD system can be expanded to calculate first and second derivatives simulataneously </li>    
</ul> 

<center> <h1> Why Julia ? </h1> </center>


<table background-color="white">
<tr></tr>
<tr> <td> <h2>JIT Just In Time compiler </h2></td> <td>  $\quad \Rightarrow \quad$ </td> <td style="text-align:center"> <h2> Julia is fast </h2></td>  </tr>
<tr></tr>
<tr><td><h2> Parametric/Dynamic type system </h2></td> <td>  $\quad \Rightarrow \quad$ </td style="text-align:center"> <td> <h2>Implementing an AD system is easy</h2></td> </tr>
<tr></tr>
<tr> <td> <h2>Multiple Dispatch </h2></td> <td>  $\quad \Rightarrow \quad$ </td> <td style="text-align:center"> <h2>No need to rewrite code</h2> </td></tr>
<!-- <tr></tr><tr> <td> <h2>Multithreading/Distributed </h2></td> <td>  $\quad \Rightarrow \quad$ </td> <td style="text-align:center"> <h2>Easily scalable code</h2> </td></tr> -->
</table>


<h1><center> AD4SM.jl </center></h1>
<ul>
<li> A second order forward mode AD system </li>
    <li> A <u>basic</u> library of elements <ul>
<li> Continous elements
<ul>
<li> 2D elments
    <ul>
        <li> Triangular    </li>
        <li> Quadrilateral </li>
    </ul>
</li>
<li> 3D elments
    <ul>
        <li> Tetrahedron    </li>
        <li> Hexahedron </li>
    </ul>
</li>
</li>
<li> Structural elments
    <ul>
        <li> Rods    </li>
        <li> Beams </li>
    </ul>
</li>
</ul> </li>
<li> Constraint equations </li>
<li> Iterative non-linear solver </li>
</ul>

<h4><center> A second order forward mode AD system </center></h4>

<p> AD4SM.jl AD follows is based on ForwardDiff.jl. </p>

<ul>
    <li>a <i>first order</i> dual number is quantity of the type:
        \begin{equation*}
    \textbf{x} \equiv x_0 + x_i\, \textbf{i}_i
        \end{equation*}
    </li>
    <li>a <i>second order</i> dual number is quantity of the type:
        \begin{equation*}
    \textbf{x} \equiv x_0 + x_i\, \textbf{i}_i + x_{ij}\, \textbf{i}_{ij}
        \end{equation*}
    </li>
</ul>

where :
-     $x_i$ are the components of the gradient 
-     $x_{ij}$ are the components of the Hessian. 


`D2` is a possible structure for storing the value, the gradient and the hessian of a quantity

```Julia
struct D2{T,N,M} <:Number 
  v::T
  g::NTuple{N,T}
  h::NTuple{M,T}
end
```

with
- `T` the type of the values
- `N` the number of independent variables controlling the gradient
- `M` = N(N+1)÷2 is the number of independent elements in the Hessian

<h4> extending operators to work with D2 type </h4>

we now need to properly extend the arithmetic opertors over the `D2` type, given two dual numbers of the type:
$$\mathbf{x} \equiv x_0 + x_i\, \mathbf{i}_i + x_{ij}\, \mathbf{i}_{ij}$$
$$\textbf{y} \equiv y_0 + y_i\, \textbf{i}_i + y_{ij}\, \textbf{i}_{ij}$$


- the sum rule is:
$$\textbf{x} + \textbf{y} = x_0+y_0 + \left(x_i+y_i\right) \textbf{i}_i + \left(x_{ij}+y_{ij}\right) \textbf{i}_{ij} \qquad \text{with} \qquad \begin{cases}i=1\dots N\\
j=1\dots i\\\end{cases}$$


- the product rule is:

$$\textbf{x}  \textbf{y} = x_0y_0 + \left(y_0 x_i+x_0 y_i\right) \textbf{i}_i + \left(x_{ij}y_0 + x_i y_j + x_j y_i + x_0 y_{ij}\right) \textbf{i}_{ij} $$

- the inverse rule is:
$$\frac{1}{\textbf{x}} = \frac{1}{x_0} - \frac{1}{x_0^2}\, x_i\, \textbf{i}_i + \left(\frac{2}{x_0^3} x_i x_j -\frac{1}{x_0} x_{ij}\right)\, \textbf{i}_{ij} $$


- the power rule is:

$$\textbf{x}^n = x_0^n + \left(n\,x_0^{n-1}\, x_i\right)\, \textbf{i}_i + \left[n\,(n-1)\,x_0^{n-2}\, x_ix_j + n\,x_0^{n-1}\, x_{ij}\right]\, \textbf{i}_{ij}$$

<h1> Some examples </h1>
<p> Simulation results of models with both geometric and material non linearities including non trivial constraints and boundary conditions </p>
    
    
<table><tr></tr>
<tr>
<td> <video controls autoplay loop src="./figures/AxSymDomainj_e.mp4" width=300/> </td>
<td> <video controls autoplay loop src="./figures/AxSymDomainj_d.mp4" width=300/>  </td>
<td>    
<video controls autoplay loop src="./figures/Pattern2D03FinerMesh02j.mp4" width=400/> 
</td>    
</tr>
</table>
<table><tr></tr>
<tr><td><center>
    <video controls autoplay loop src="./figures/example05c.mp4" width=1000/>
    </center></td></tr>
</table>

<p>
more details in 
<p> <b> Automatic Differentiation for Solid Mechanics </b>. Vigliotti, A., Auricchio, F. <i>Arch Computat Methods Eng</i> <b>28</b>, 875–895 (2021) </p>
</p>

<p> Simulation results of models with both geometric and material non linearities including non trivial constraints and boundary conditions </p>
        
<table><tr></tr>
<tr>
<td> <video controls autoplay loop src="./figures/AxSymDomainj_d.mp4" width=500/>  </td>
<td>    
<video controls autoplay loop src="./figures/Pattern2D03FinerMesh02j.mp4" width=625/> 
</td>    
</tr>
</table>
<table><tr></tr>
<tr><td><center>
    <video controls autoplay loop src="./figures/example05c.mp4" width=900/>
    </center></td></tr>
</table>

<p>
more details in 
<p> <b> Automatic Differentiation for Solid Mechanics </b>. Vigliotti, A., Auricchio, F. <i>Arch Computat Methods Eng</i> <b>28</b>, 875–895 (2021) </p>
</p>

<h2> Fracture toughness of Ceramic Matrix Composites </h2>


\begin{equation*}
    U(u,d) = \phi^{-} + (1-d^2)\, \phi^{+} + \frac{G_c}{2}\left(d^2 + d_{,i}d_{,i}\right)
\end{equation*}


<table>
    <tr></tr>
<tr><td> <video controls autoplay loop src="./figures/cn_central_matrix_2x2_r500L1596lc0075lcf3000tht00mu0050rho0005tmov.mp4" width=600> </td>
<td> <video controls autoplay loop src="./figures/cn_central_matrix_2x2_r500L1596lc0075lcf3000tht00mu0250rho0005tmov.mp4" width=600> </td>    </tr>
</table>
<table>
    <tr></tr>
<tr><td> <video controls autoplay loop src="./figures/cn_central_matrix_2x2_r500L1596lc0075lcf3000tht00pdd0001cmov.mp4" width=600> </td>
<td> <video controls autoplay loop src="./figures/cn_central_matrix_2x2_r500L1596lc0075lcf3000tht00pdd0050cmov.mp4" width=600> </td>    </tr>
</table>

<center>
<h1> Thank you!  </h1>

<h3> a.vigliotti@cira.it </h3>
</center>

140_000 nodes, 420_000 DoFs, 15 sec per iteration